# Anti-CRISPR Exp15~Exp17: ProteinBERT + PSSM 融合

**功能**：本 notebook 总览与适用范围（PSSM 流水线 + 融合实验 + 基线/消融对照）。

该 notebook 聚焦 `pssm融合后续实验计划_5b0b33fd.plan.md`：
- PSSM 数据生产流水线（UniRef50 + PSI-BLAST）
- RPSSM(110) + PSSM-AC(200) + PSSM-composition(400) + DPC-PSSM(400)
- Exp15/Exp16/Exp17 主实验与关键消融（固定 seeds）
- 与纯 ProteinBERT 基线同协议、同 seeds 对照

In [1]:
# 功能：导入依赖并设置项目路径、特征缓存路径与固定随机种子
import os
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score, brier_score_loss, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import train_test_split

from proteinbert import (
    OutputType,
    OutputSpec,
    FinetuningModelGenerator,
    load_pretrained_model,
    finetune,
    FusionTrainConfig,
    load_anticrispr_with_ids,
    load_feature_cache,
    attach_pssm_features,
    run_finetune_with_pssm,
)
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

PROJECT_ROOT = '/home/nemophila/projects/protein_bert'
BENCHMARKS_DIR = f'{PROJECT_ROOT}/anticrispr_benchmarks'
WORK_ROOT = f'{PROJECT_ROOT}/pssm_work'
FEATURE_VARIANTS = ['310', '710', '1110']
SEEDS = [0, 11, 22, 33, 44]


/home/nemophila/miniconda3/envs/tf24pb/lib/python3.8/site-packages/joblib/_multiprocessing_helpers.py:46: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


2026-02-13 14:54:00.548917: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


## 阶段0-1：PSSM 数据准备（命令级）

**功能**：说明在命令行完成 PSSM 数据准备（下载/建库、生成 FASTA、跑 PSI-BLAST、特征提取、建缓存）的步骤。

首次执行（数据库下载与建库）：

```bash
cd /home/nemophila/projects/protein_bert
conda install -c bioconda blast -y
bash scripts/pssm/00_download_uniref50.sh
mkdir -p /home/nemophila/projects/protein_bert/pssm_work/{fasta,pssm,logs,features}
```

生成 FASTA + 跑 PSI-BLAST + 提特征：

```bash
python scripts/pssm/00_prepare_fasta.py \
  --train-csv /home/nemophila/projects/protein_bert/anticrispr_benchmarks/anticrispr_binary.train.csv \
  --test-csv /home/nemophila/projects/protein_bert/anticrispr_benchmarks/anticrispr_binary.test.csv \
  --work-root /home/nemophila/projects/protein_bert/pssm_work

bash scripts/pssm/01_run_psiblast_batch.sh \
  /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  /home/nemophila/projects/protein_bert/blast_db/uniref50 8

bash scripts/pssm/02_retry_failed.sh \
  /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  /home/nemophila/projects/protein_bert/pssm_work/logs/failed_ids.txt \
  /home/nemophila/projects/protein_bert/blast_db/uniref50 4

python scripts/pssm/03_extract_rpssm_pssmac.py \
  --manifest-csv /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  --work-root /home/nemophila/projects/protein_bert/pssm_work

python scripts/pssm/04_build_feature_cache.py \
  --manifest-csv /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  --work-root /home/nemophila/projects/protein_bert/pssm_work \
  --variants 310,710,1110
```

In [2]:
# 功能：加载 Anti-CRISPR 数据并按 sample_id 挂载 310/710/1110 三套 PSSM 特征
train_base_df, test_base_df = load_anticrispr_with_ids(BENCHMARKS_DIR, benchmark_name='anticrispr_binary')

variant_datasets = {}
for variant in FEATURE_VARIANTS:
    parquet_path = f'{WORK_ROOT}/features/pssm_features_{variant}.parquet'
    csv_path = f'{WORK_ROOT}/features/pssm_features_{variant}.csv'
    cache_path = parquet_path if os.path.exists(parquet_path) else csv_path
    feature_df, feature_cols = load_feature_cache(cache_path)

    train_df_v = attach_pssm_features(train_base_df, feature_df, feature_cols)
    test_df_v = attach_pssm_features(test_base_df, feature_df, feature_cols)
    variant_datasets[variant] = (train_df_v, test_df_v, feature_cols)

    print(f'variant={variant} train shape:', train_df_v.shape)
    print(f'variant={variant} test shape:', test_df_v.shape)
    print(f'variant={variant} feature dim:', len(feature_cols))

train_df_310, test_df_310, feature_cols_310 = variant_datasets['310']
train_df_710, test_df_710, feature_cols_710 = variant_datasets['710']
train_df_1110, test_df_1110, feature_cols_1110 = variant_datasets['1110']


variant=310 train shape: (1107, 313)
variant=310 test shape: (286, 313)
variant=310 feature dim: 310


variant=710 train shape: (1107, 713)
variant=710 test shape: (286, 713)
variant=710 feature dim: 710


variant=1110 train shape: (1107, 1113)
variant=1110 test shape: (286, 1113)
variant=1110 feature dim: 1110


In [3]:
# 功能：定义二分类评估指标（AUC/AUPRC/F1/MCC/Brier/ECE）及验证集最优阈值搜索
def expected_calibration_error(y_true, y_prob, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ids = np.digitize(y_prob, bins) - 1
    ece = 0.0
    n = len(y_true)
    for b in range(n_bins):
        m = ids == b
        if np.any(m):
            conf = float(np.mean(y_prob[m]))
            acc = float(np.mean(y_true[m]))
            ece += (np.sum(m) / n) * abs(acc - conf)
    return float(ece)

def evaluate_binary(y_true, y_prob, thr=0.5):
    y_cls = (y_prob >= thr).astype(int)
    return {
        'AUC': float(roc_auc_score(y_true, y_prob)),
        'AUPRC': float(average_precision_score(y_true, y_prob)),
        'F1': float(f1_score(y_true, y_cls)),
        'MCC': float(matthews_corrcoef(y_true, y_cls)),
        'Brier': float(brier_score_loss(y_true, y_prob)),
        'ECE': float(expected_calibration_error(y_true, y_prob, n_bins=10)),
        'Threshold': float(thr),
    }

def find_best_thr(y_true, y_prob):
    best_thr, best_f1 = 0.5, -1.0
    for thr in np.linspace(0.05, 0.95, 19):
        cur_f1 = f1_score(y_true, (y_prob >= thr).astype(int))
        if cur_f1 > best_f1:
            best_f1, best_thr = cur_f1, float(thr)
    return best_thr


In [4]:
# 功能：单 seed 下训练并评估纯 ProteinBERT（多层 hidden/global 拼接）基线，在验证集选阈值后在测试集打分
def run_baseline_one_seed(seed):
    sub_train, sub_valid = train_test_split(
        train_df_310[['seq', 'label']], test_size=0.1, stratify=train_df_310['label'], random_state=seed
    )

    output_type = OutputType(False, 'binary')
    output_spec = OutputSpec(output_type, [0, 1])

    pretrained_model_generator, input_encoder = load_pretrained_model(
        local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True,
        validate_downloading=False,
    )

    mg = FinetuningModelGenerator(
        pretrained_model_generator,
        output_spec=output_spec,
        pretraining_model_manipulation_function=get_model_with_hidden_layers_as_outputs,
        dropout_rate=0.4,
        head_type='two_layer',
        loss_type='bce',
        lr=2e-5,
    )

    finetune(
        mg,
        input_encoder,
        output_spec,
        sub_train['seq'],
        sub_train['label'],
        sub_valid['seq'],
        sub_valid['label'],
        seq_len=512,
        batch_size=8,
        max_epochs_per_stage=8,
        begin_with_frozen_pretrained_layers=True,
        n_final_epochs=0,
    )

    model = mg.create_model(512)
    X_valid = input_encoder.encode_X(sub_valid['seq'].tolist(), 512)
    valid_prob = model.predict(X_valid, batch_size=8, verbose=0).reshape(-1)
    thr = find_best_thr(sub_valid['label'].to_numpy(), valid_prob)

    X_test = input_encoder.encode_X(test_df_310['seq'].tolist(), 512)
    test_prob = model.predict(X_test, batch_size=8, verbose=0).reshape(-1)
    return evaluate_binary(test_df_310['label'].to_numpy(), test_prob, thr=thr)


In [5]:
# 功能：固定 seed 下跑基线与 Exp15/16/17 融合实验，汇总为 exp_df
cfg = FusionTrainConfig(
    seq_len=512,
    batch_size=8,
    frozen_epochs=6,
    unfrozen_epochs=12,
    frozen_lr=1e-4,
    unfrozen_lr=2e-5,
    pssm_dropout=0.3,
    global_dropout=0.3,
    pssm_hidden_dim=128,
    global_hidden_dim=128,
    global_bottleneck_dim=64,
    fusion_hidden_dim=128,
    use_hidden_global_concat=True,
)

all_rows = []
for seed in SEEDS:
    print(f'Running seed={seed}')

    base_metrics = run_baseline_one_seed(seed)
    all_rows.append({'Exp': 'Baseline_ProteinBERT', 'Seed': seed, **base_metrics})

    pretrained_model_generator, input_encoder = load_pretrained_model(
        local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True,
        validate_downloading=False,
    )
    rpssm_metrics = run_finetune_with_pssm(
        pretrained_model_generator,
        input_encoder,
        train_df_310,
        test_df_310,
        feature_cols=feature_cols_310[:110],
        seed=seed,
        cfg=cfg,
    )
    all_rows.append({'Exp': 'Ablation_RPSSM_110', 'Seed': seed, **rpssm_metrics})

    pretrained_model_generator, input_encoder = load_pretrained_model(
        local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True,
        validate_downloading=False,
    )
    exp15_metrics = run_finetune_with_pssm(
        pretrained_model_generator,
        input_encoder,
        train_df_310,
        test_df_310,
        feature_cols=feature_cols_310,
        seed=seed,
        cfg=cfg,
    )
    all_rows.append({'Exp': 'Exp15_ProteinBERT_PSSM310', 'Seed': seed, **exp15_metrics})

    pretrained_model_generator, input_encoder = load_pretrained_model(
        local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True,
        validate_downloading=False,
    )
    exp16_metrics = run_finetune_with_pssm(
        pretrained_model_generator,
        input_encoder,
        train_df_710,
        test_df_710,
        feature_cols=feature_cols_710,
        seed=seed,
        cfg=cfg,
    )
    all_rows.append({'Exp': 'Exp16_ProteinBERT_PSSM710', 'Seed': seed, **exp16_metrics})

    pretrained_model_generator, input_encoder = load_pretrained_model(
        local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True,
        validate_downloading=False,
    )
    exp17_metrics = run_finetune_with_pssm(
        pretrained_model_generator,
        input_encoder,
        train_df_1110,
        test_df_1110,
        feature_cols=feature_cols_1110,
        seed=seed,
        cfg=cfg,
    )
    all_rows.append({'Exp': 'Exp17_ProteinBERT_PSSM1110', 'Seed': seed, **exp17_metrics})

exp_df = pd.DataFrame(all_rows)
exp_df


Running seed=0
[2026_02_13-14:54:02] Training set: Filtered out 0 of 996 (0.0%) records of lengths exceeding 510.


[2026_02_13-14:54:02] Validation set: Filtered out 0 of 111 (0.0%) records of lengths exceeding 510.


[2026_02_13-14:54:02] Training with frozen pretrained layers...


2026-02-13 14:54:02.953478: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2026-02-13 14:54:02.955712: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2026-02-13 14:54:02.962199: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2026-02-13 14:54:02.962227: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2026-02-13 14:54:02.962919: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-13 14:54:02.975631: I tensorflow/compiler/jit/xla_gpu_d

2026-02-13 14:54:04.483210: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2026-02-13 14:54:04.484142: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2500000000 Hz


...

123/125 [============================>.] - ETA: 0s - loss: 0.0980

124/125 [============================>.] - ETA: 0s - loss: 0.0982

125/125 [==============================] - ETA: 0s - loss: 0.0982

125/125 [==============================] - 26s 208ms/step - loss: 0.0982 - val_loss: 0.5125


,Exp,Seed,AUC,AUPRC,F1,MCC,Brier,ECE,Threshold
0,Baseline_ProteinBERT,0,0.884911,0.557779,0.472727,0.417580,0.065615,0.056012,0.35
1,Ablation_RPSSM_110,0,0.896746,0.631171,0.509091,0.457872,0.056186,0.029678,0.35
2,Exp15_ProteinBERT_PSSM310,0,0.918935,0.670580,0.583333,0.547723,0.051313,0.044551,0.45
3,Exp16_ProteinBERT_PSSM710,0,0.922041,0.651275,0.600000,0.560279,0.060390,0.052736,0.50
4,Exp17_ProteinBERT_PSSM1110,0,0.932988,0.655719,0.558140,0.537771,0.052534,0.042970,0.60
5,Baseline_ProteinBERT,11,0.901923,0.623558,0.480000,0.437234,0.057763,0.039445,0.20
6,Ablation_RPSSM_110,11,0.911834,0.621427,0.526316,0.494889,0.061465,0.062312,0.25
7,Exp15_ProteinBERT_PSSM310,11,0.912278,0.607807,0.588235,0.548072,0.056160,0.037794,0.45
8,Exp16_ProteinBERT_PSSM710,11,0.924704,0.723811,0.697674,0.692088,0.047446,0.043822,0.65
9,Exp17_ProteinBERT_PSSM1110,11,0.914349,0.677833,0.588235,0.548072,0.062150,0.051368,0.60


In [6]:
# 功能：按实验组汇总各指标均值与标准差，按 AUPRC 排序
summary = exp_df.groupby('Exp')[['AUC','AUPRC','F1','MCC','Brier','ECE']].agg(['mean','std'])
summary = summary.sort_values(('AUPRC', 'mean'), ascending=False)
summary


AUC               AUPRC                  F1  \
                                mean       std      mean       std      mean   
Exp                                                                            
Exp17_ProteinBERT_PSSM1110  0.923491  0.011133  0.679483  0.018050  0.551497   
Exp16_ProteinBERT_PSSM710   0.915089  0.012354  0.667929  0.032698  0.571798   
Exp15_ProteinBERT_PSSM310   0.911893  0.008642  0.636605  0.030797  0.550801   
Ablation_RPSSM_110          0.897929  0.012892  0.624106  0.014972  0.481342   
Baseline_ProteinBERT        0.881893  0.013832  0.588669  0.023985  0.469075   

                                           MCC               Brier            \
                                 std      mean       std      mean       std   
Exp                                                                            
Exp17_ProteinBERT_PSSM1110  0.023920  0.525306  0.027215  0.055713  0.004766   
Exp16_ProteinBERT_PSSM710   0.086680  0.538262  0.100446  0.054832  0.005281   
Exp15_ProteinBERT_PSSM310   0.050670  0.511201  0.057018  0.055608  0.004121   
Ablation_RPSSM_110          0.043521  0.442173  0.038139  0.059502  0.003644   
Baseline_ProteinBERT        0.020578  0.418230  0.024319  0.059167  0.003757   

                                 ECE            
                                mean       std  
Exp                                             
Exp17_ProteinBERT_PSSM1110  0.046528  0.012405  
Exp16_ProteinBERT_PSSM710   0.048171  0.004058  
Exp15_ProteinBERT_PSSM310   0.044594  0.006122  
Ablation_RPSSM_110          0.047375  0.014279  
Baseline_ProteinBERT        0.046577  0.006130

In [7]:
# 功能：将实验结果明细与汇总表保存为 CSV
os.makedirs(f'{WORK_ROOT}/features', exist_ok=True)
report_path = f'{WORK_ROOT}/features/exp_results.csv'
summary_path = f'{WORK_ROOT}/features/exp_summary.csv'
exp_df.to_csv(report_path, index=False)
summary.to_csv(summary_path)
print('saved:', report_path)
print('saved:', summary_path)


saved: /home/nemophila/projects/protein_bert/pssm_work/features/exp_results.csv
saved: /home/nemophila/projects/protein_bert/pssm_work/features/exp_summary.csv
